In [26]:
# STEP 0: Setup for Google Colab
!pip install ipywidgets --quiet

# ✅ NEW: Clone your GitHub repo and navigate into it
import os, sys, shutil

# Remove old copy if it exists
if os.path.exists("RealOrAI-DogDetector"):
    shutil.rmtree("RealOrAI-DogDetector")

!git clone https://github.com/zhaoq23/RealOrAI-DogDetector.git

# Change directory to repo and add to Python path
os.chdir("RealOrAI-DogDetector")
sys.path.append(".")

# ✅ Now your model import will work
import torch
from IPython.display import display
import ipywidgets as widgets
from models import BayesianMLP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Cloning into 'RealOrAI-DogDetector'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 141 (delta 38), reused 10 (delta 2), pack-reused 45 (from 1)
Receiving objects: 100% (141/141), 105.86 MiB | 26.19 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [27]:
!pip install ipywidgets --quiet

import torch
from torch.utils.data import DataLoader
from IPython.display import display
import ipywidgets as widgets
from models import BayesianMLP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [28]:
def load_model(model_path="plusdiff_model.pt"):
    model = BayesianMLP()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model


In [29]:
def classify_feature_tensor(tensor, model):
    tensor = tensor.to(device)
    with torch.no_grad():
        output = model(tensor)
        probs = torch.sigmoid(output).squeeze()

        # If only one item, make it iterable
        if probs.dim() == 0:
            probs = probs.unsqueeze(0)

        for i, p in enumerate(probs):
            label = "AI-GENERATED 🤖" if p >= 0.5 else "REAL 🐶"
            print(f"Image {i+1}: {label}  |  Probability: {p:.2f}")


In [30]:
upload = widgets.FileUpload(accept='.pt', multiple=False)

def on_upload_change(change):
    if upload.value:
        file_info = next(iter(upload.value.values()))
        with open("temp.pt", "wb") as f:
            f.write(file_info['content'])

        # Load the .pt file and extract features
        data = torch.load("temp.pt")
        if isinstance(data, tuple):  # (features, label)
            tensor = data[0]
        else:
            tensor = data

        if len(tensor.shape) == 1:
            tensor = tensor.unsqueeze(0)  # Add batch dimension if needed

        # ✅ Add this padding fix
        if tensor.shape[1] == 384:
            padding = torch.zeros((tensor.shape[0], 1))
            tensor = torch.cat([tensor, padding], dim=1)

        # Load model and classify
        model = load_model()
        classify_feature_tensor(tensor, model)

upload.observe(on_upload_change, names='value')
display(upload)


FileUpload(value={}, accept='.pt', description='Upload')

Image 1: AI-GENERATED 🤖  |  Probability: 1.00
Image 2: AI-GENERATED 🤖  |  Probability: 1.00
Image 3: AI-GENERATED 🤖  |  Probability: 1.00
Image 4: AI-GENERATED 🤖  |  Probability: 1.00
Image 5: AI-GENERATED 🤖  |  Probability: 1.00
Image 6: AI-GENERATED 🤖  |  Probability: 1.00
Image 7: AI-GENERATED 🤖  |  Probability: 1.00
Image 8: AI-GENERATED 🤖  |  Probability: 1.00
Image 9: AI-GENERATED 🤖  |  Probability: 1.00
Image 10: AI-GENERATED 🤖  |  Probability: 1.00
Image 11: AI-GENERATED 🤖  |  Probability: 0.97
Image 12: AI-GENERATED 🤖  |  Probability: 1.00
Image 13: AI-GENERATED 🤖  |  Probability: 1.00
Image 14: AI-GENERATED 🤖  |  Probability: 1.00
Image 15: AI-GENERATED 🤖  |  Probability: 1.00
Image 16: AI-GENERATED 🤖  |  Probability: 0.92
Image 17: AI-GENERATED 🤖  |  Probability: 1.00
Image 18: AI-GENERATED 🤖  |  Probability: 1.00
Image 19: AI-GENERATED 🤖  |  Probability: 1.00
Image 20: AI-GENERATED 🤖  |  Probability: 1.00
Image 21: AI-GENERATED 🤖  |  Probability: 0.92
Image 22: AI-GENERATED